# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.



In [75]:
import pandas as pd
import numpy as nd



credit_score = pd.read_csv("/datasets/credit_scoring_eng.csv")
print(credit_score.head(55))


    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42    bachelor's degree             0   
1          1   -4024.803754         36  secondary education             1   
2          0   -5623.422610         33  Secondary Education             1   
3          3   -4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
5          0    -926.185831         27    bachelor's degree             0   
6          0   -2879.202052         43    bachelor's degree             0   
7          0    -152.779569         50  SECONDARY EDUCATION             1   
8          2   -6929.865299         35    BACHELOR'S DEGREE             0   
9          0   -2188.756445         41  secondary education             1   
10         2   -4171.483647         36    bachelor's degree             0   
11         0    -792.701887         40  secondary education             1   

## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

[Now let's explore our data. You'll want to see how many columns and rows it has, look at a few rows to check for potential issues with the data.]

In [76]:
# Let's see how many rows and columns our dataset has
credit_score.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


The info method provides a good display of number of rows: (21525) and columns: (12)

In [77]:
# let's print the first N rows
print(credit_score.head(10))


   children  days_employed  dob_years            education  education_id  \
0         1   -8437.673028         42    bachelor's degree             0   
1         1   -4024.803754         36  secondary education             1   
2         0   -5623.422610         33  Secondary Education             1   
3         3   -4124.747207         32  secondary education             1   
4         0  340266.072047         53  secondary education             1   
5         0    -926.185831         27    bachelor's degree             0   
6         0   -2879.202052         43    bachelor's degree             0   
7         0    -152.779569         50  SECONDARY EDUCATION             1   
8         2   -6929.865299         35    BACHELOR'S DEGREE             0   
9         0   -2188.756445         41  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1

[Describe what you see and notice in your printed data sample. Are there any issues that may need further investigation and changes?]

-The data could be organized in a more concise way that includes categories which would help make it more digestable. We can possibly organize data based on categories of purpose, for example.
-The negative sign displayed on values in the days_employed column might also be problematic.
-There are several values in the ["education"] column that contain uppercase and lowercase letters, which also should be fixed.

In [78]:
# Get info on data
credit_score.info()
print()
print(credit_score["total_income"].isna())
print()
print(credit_score["days_employed"].isna())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

0        False
1        False
2        False
3        False
4        False
         ...  
21520    False
21521    False
21522    False
21523    False
21524    False
Name:

[Are there missing values across all columns or just a few? Briefly describe what you see in 1-2 sentences.]
Missing columns are observed accross two columns: "total_income" and ""days_employed", which contain 2174 missing rows. Also, there in all the rows where days_employed = NaN, total_income also= "NaN. So, there seems to be some sort of pattern here.

In [79]:
# Let's look at the filtered table with missing values in the the first column with missing data
print(credit_score[credit_score['total_income'].isna()])



       children  days_employed  dob_years            education  education_id  \
12            0            NaN         65  secondary education             1   
26            0            NaN         41  secondary education             1   
29            0            NaN         63  secondary education             1   
41            0            NaN         50  secondary education             1   
55            0            NaN         54  secondary education             1   
...         ...            ...        ...                  ...           ...   
21489         2            NaN         47  Secondary Education             1   
21495         1            NaN         50  secondary education             1   
21497         0            NaN         48    BACHELOR'S DEGREE             0   
21502         1            NaN         42  secondary education             1   
21510         2            NaN         28  secondary education             1   

           family_status  family_status

[Do missing values seem symmetric? Can we be sure in this assumption? Explain your thoughts briefly in this section. You may probably want to conduct further investigations, and count the missing values in all the rows with missing values to confirm the the missing samples are of the same size.]
The missing values between the two missing columns do seem symmetric, which appears to be reflected in the boolean values listed. In addition, both columns generate the exact same number of missing values (shown below). I see that most rows containing NaN also have common education id=1 (secondary education)

In [80]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.
#print(credit_score.isna().sum())
#print(credit_score[credit_score.duplicated(subset='days_employed')])
#missing_values = credit_score[credit_score["total_income"]=="NaN"]
#print(missing_values)
print(credit_score['total_income'].isna().sum())
print(credit_score["days_employed"].isna().sum())
credit_score.loc[(credit_score['days_employed'].isna()),:]
credit_score.loc[(credit_score['total_income'].isna()),:]

2174
2174


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Intermediate conclusion**

[Does the number of rows in the filtered table match the number of missing values? What conclusion can we make from this?]

[Calculate the percentage of the missing values compared to the whole dataset. Is it a considerably large piece of data? If so, you may want to fill the missing values. To do that, firstly we should consider whether the missing data could be due to the specific client characteristic, such as employment type or something else. You will need to decide which characteristic *you* think might be the reason. Secondly, we should check whether there's any dependence missing values have on the value of other indicators with the columns with identified specific client characteristic.]

[Explain your next steps and how they correlate with the conclusions you made so far.]
 The number of rows in the filtered table match the number of missing values: 2174. We can conclude that the missing values are not random, but may be associated with some other reason.The missing data in ["days_employed"] and in ["total_income"] do not appear to be a large piece of data, as both cumulatively amount to about 20% of the data. The proportion of missing values is not large enough to drop the columns, so this means that I will need to fill them. I think that the missing data in both columns might have something to due with income type, as I noticed that there were several "NaN" in rows where income type was "retired". Both columns that contain the missing values are numeric and do not have an direct dependencce on other indicators with columns with education. If a client's highest education is secondary education

In [81]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values
#print(credit_score[credit_score["education"] != "secondary education"])

#print()

#print(credit_score.loc[(credit_score["education_id"] != int("1")) & (credit_score["days_employed"] == float("NaN")),:])
            
(credit_score.loc[(credit_score["education_id"] == int("1")) & (credit_score["total_income"].isna()),:])
print(credit_score.loc[(credit_score["debt"] == int("0")) & (credit_score["total_income"].isna()),:])

       children  days_employed  dob_years            education  education_id  \
12            0            NaN         65  secondary education             1   
26            0            NaN         41  secondary education             1   
29            0            NaN         63  secondary education             1   
41            0            NaN         50  secondary education             1   
65            0            NaN         21  secondary education             1   
...         ...            ...        ...                  ...           ...   
21489         2            NaN         47  Secondary Education             1   
21495         1            NaN         50  secondary education             1   
21497         0            NaN         48    BACHELOR'S DEGREE             0   
21502         1            NaN         42  secondary education             1   
21510         2            NaN         28  secondary education             1   

           family_status  family_status

In [82]:
# Checking distribution
print(credit_score.isna().sum() / credit_score.shape[0])

print()

print(credit_score["children"].value_counts())
#Is -1 children a typo? Should we replace this with "1" child or should we drop the whole row?
print(credit_score["family_status"].value_counts())
print(credit_score.groupby("family_status")["family_status_id"].count())
print(credit_score["education"].value_counts())
print(credit_score["purpose"].value_counts())
print(credit_score["dob_years"].value_counts())
print(credit_score["education_id"].value_counts())
print()


      



children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64
family_status
civil partnership     4177
divorced              1195
married              12380
unmarried             2813
widow / widower        960
Name: family_status_id, dtype: int64
secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BAC

[Describe your findings here.]

**Possible reasons for missing values in data**

[Propose your ideas on why you think the values might be missing. Do you think they are missing randomly or there are any patterns?]
It appears that where missing values are present in the ["days_employed"] column, there also appears to be missing values in the ["total_income"] column as well.
[Let's start checking whether the missing values are random.]

-It seems that in the majority of rows where missing values are observed in both columns, the debt=0, which might be it might be correlated.
-10 of the NaN rows [dob_years] = 0, which doesn't make sense, so that data should probably be discarded


In [83]:
# Checking the distribution in the whole dataset
print(credit_score["family_status"].value_counts())
print()
print(credit_score["dob_years"].value_counts())
print()
print(credit_score.describe())


married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

           children  days_employed     dob_years  education_id  \
count  21525.000000   19351.000000  21525.000000  21525.000000   
mean       0.538908   63046.497661     43.293380      0.817236   
std        1.381587  

**Intermediate conclusion**

[Is the distribution in the original dataset similar to the distribution of the filtered table? What does that mean for us?]
The distribution in the original dataset is mostly skewed to the right in regards to number of children, education_id, debt, total_income, which is similar to the distribution shown in the filtered table. This probably means that there is some sort of association between these client characteristics. Essentially, they most likely are correlated with each other in some way, but I am not quite sure how just yet. This probably means that the missing values are not random, but are due to some other reason. Also, I noticed that there are 10 rows where [total_income] and [days_employed]=="NaN and [dob_years] == 0. It does not make sense for a client to be 0 years of age and be applying for a loan. This could also be contributing to the misisng values. 2004 of the 2174 rows of missing data have a [debt] value of 0. This is something to also consider.

[If you think we can't make any conclusions yet, let's investigate our dataset further. Let's think about other reasons that could lead to data missing and check if we can find any patterns that may lead us to thinking that the missing values are not random. Because this is your work, this is section is optional.]

In [84]:
# Check for other reasons and patterns that could lead to missing values

print(credit_score.loc[(credit_score["dob_years"] < int("18")) & (credit_score["total_income"].isna()),:])


       children  days_employed  dob_years            education  education_id  \
1890          0            NaN          0    bachelor's degree             0   
2284          0            NaN          0  secondary education             1   
4064          1            NaN          0  secondary education             1   
5014          0            NaN          0  secondary education             1   
6411          0            NaN          0    bachelor's degree             0   
6670          0            NaN          0    Bachelor's Degree             0   
8574          0            NaN          0  secondary education             1   
12403         3            NaN          0  secondary education             1   
13741         0            NaN          0  secondary education             1   
19829         0            NaN          0  secondary education             1   

           family_status  family_status_id gender income_type  debt  \
1890           unmarried                 4      

**Intermediate conclusion**

[Can we finally confirm that missing values are accidental? Check for anything else that you think might be important here.]
While most of the missing data is correlated with a debt = 0, having a debt=0 is not likely to be the cause of the missing data because there does not seem to be a direct association between have no debt and having missing information in those areas. I checked all other client characteristics, and none of them showed to be associated or correlated with the number of rows of misisng values. Overall, the missing values do not seem to be directly associated with any particular other column. However, 10 rows of missing data have a value of 0 for ["dob_years"]. Since you have to be 18 years or older to apply for a loan, this could be a legitamite reason why we are missing data in these rows. 

In [85]:
# Checking for other patterns - explain which
#print(credit_score.loc[(credit_score["children"] < int("4")) & (credit_score["total_income"].isna()),:])
#print(credit_score.loc[(credit_score["children"] < int("2")) & (credit_score["total_income"].isna()),:])
print(credit_score[(credit_score["education_id"] != 1) & (credit_score["days_employed"].isnull())])

       children  days_employed  dob_years          education  education_id  \
67            0            NaN         52  bachelor's degree             0   
72            1            NaN         32  bachelor's degree             0   
82            2            NaN         50  bachelor's degree             0   
90            2            NaN         35  bachelor's degree             0   
94            1            NaN         34  bachelor's degree             0   
...         ...            ...        ...                ...           ...   
21281         1            NaN         30  bachelor's degree             0   
21294         0            NaN         45  BACHELOR'S DEGREE             0   
21432         1            NaN         38       some college             2   
21463         1            NaN         35  bachelor's degree             0   
21497         0            NaN         48  BACHELOR'S DEGREE             0   

           family_status  family_status_id gender    income_typ

**Conclusions**

[Did you find any patterns? How did you come to this conclusion?]
Here, I checked for any patterns between the rows of missing data and number of children. I concluded that there does not seem to be any apparent pattern between the number of children and missing data in the columns.

[Explain how you will address the missing values. Consider the categories in which values are missing.]
To address the missing values, I will not fill them in right away with a mean() for example, because total_income and days_employed will vary greatly depending on each individual client and their financial situation. In addition, total monthly income is dependent other catagorical columns in this dataset like income_type (occupation). Dropping the missing values in these columns would mean getting rid of about 20% of the data, which in my opinion is too much data to sacrifice in this analysis. Thus, I will attempt to find the mean/median of both columns and decide which would be a better fit to fill in the missing values.
[Briefly plan your next steps for transforming data. You will probably need to address different types of issues: duplicates, different registers, incorrect artifacts, and missing values.]
Missing values-Find mean/median for each column
Duplicates-No duplicates found
Incorrect artifacts-Need to make all values in [education] lowercase & need to remove 10 rows where [dob_years]= 0

## Data transformation

[Let's go through each column to see what issues we may have in them.]

[Begin with removing duplicates and fixing educational information if required.]

In [86]:
# Let's see all values in education column to check if and what spellings will need to be fixed
print(credit_score["education"])
print(credit_score.head(50))

0          bachelor's degree
1        secondary education
2        Secondary Education
3        secondary education
4        secondary education
                ...         
21520    secondary education
21521    secondary education
21522    secondary education
21523    secondary education
21524    secondary education
Name: education, Length: 21525, dtype: object
    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42    bachelor's degree             0   
1          1   -4024.803754         36  secondary education             1   
2          0   -5623.422610         33  Secondary Education             1   
3          3   -4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
5          0    -926.185831         27    bachelor's degree             0   
6          0   -2879.202052         43    bachelor's degree             0   
7          0    -15

In [87]:
# Fix the registers if required
credit_score["education"] = credit_score["education"].str.lower()
print(credit_score["education"])
print(credit_score.head(30))


0          bachelor's degree
1        secondary education
2        secondary education
3        secondary education
4        secondary education
                ...         
21520    secondary education
21521    secondary education
21522    secondary education
21523    secondary education
21524    secondary education
Name: education, Length: 21525, dtype: object
    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42    bachelor's degree             0   
1          1   -4024.803754         36  secondary education             1   
2          0   -5623.422610         33  secondary education             1   
3          3   -4124.747207         32  secondary education             1   
4          0  340266.072047         53  secondary education             1   
5          0    -926.185831         27    bachelor's degree             0   
6          0   -2879.202052         43    bachelor's degree             0   
7          0    -15

In [88]:
# Checking all the values in the column to make sure we fixed them
print(credit_score["education"])


0          bachelor's degree
1        secondary education
2        secondary education
3        secondary education
4        secondary education
                ...         
21520    secondary education
21521    secondary education
21522    secondary education
21523    secondary education
21524    secondary education
Name: education, Length: 21525, dtype: object


[Check the data the `children` column]

In [89]:
# Let's see the distribution of values in the `children` column
print(credit_score["children"].value_counts())

      

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


[Are there any strange things in the column? If yes, how high is the percentage of problematic data? How could they have occurred? Make a decision on what you will do with this data and explain you reasoning.]
Yes, there is one value that is equal to -1 children. This does not make sense. Since there are only 47/21525 rows with this erroneous value of children, it accounts for less than 1% of the data in the entire dataset. I think that this might have likely been due to error and that perhaps the intention was to assign the value as "1" instead of "-1". Thus, for value "-1" in the ["children"] column, I will simply change to "1" instead. 

In [90]:
# [fix the data based on your decision]
#credit_score["children"]= credit_score["children"].drop()
#neg = credit_score[credit_score['children'] == int("-1")] 
#credit_score[credit_score["children"]].drop(neg.index, inplace=True)

credit_score.loc[credit_score['children'] == int("-1"), 'children'] = int("1")



***Note to reviewer: Thank you, I will take note of this method as well.

***Note to reviewer: I will keep this in mind for next time. 

In [91]:
# Checking the `children` column again to make sure it's all fixed
print(credit_score["children"].value_counts())


0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


All -1 values for ["children"] have been added to the 1 value

[Check the data in the `days_employed` column. Firstly think about what kind of issues could there be and what you may want to check and how you will do it.]
Key issues:
-There are several rows where data is missing.
-There are negative values for which its positive value might indicate how many days they are unemployed.
Plan:
I will change all negative values in ["days_employed"] column to positive values using .abs() method.
Once I have determined which measure of central tendancy (median/mean) best represents the column, I will use it to fill in missing values.
Finally, I will convert days in ["days_employed"] to months and create a new column called ["months_employed"], so that the column is more easy to compare.

In [92]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage
print(credit_score["days_employed"].value_counts())
credit_score.loc[(credit_score['days_employed'].isna())]


-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,secondary education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,bachelor's degree,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


[If the amount of problematic data is high, it could've been due to some technical issues. We may probably want to propose the most obvious reason why it could've happened and what the correct data might've been, as we can't drop these problematic rows.]


In [93]:
# Address the problematic values, if they exist
credit_score["days_employed"]=credit_score["days_employed"].abs()
credit_score.info()
credit_score["days_employed"].isna().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


2174

In [94]:
# Check the result - make sure it's fixed
print(credit_score["days_employed"])
def days_to_months(val):
    val = val / 30
    return val

  

print(days_to_months(4124))       
print(days_to_months(5623))
print(days_to_months(343937))

print()

credit_score["months_employed"] = credit_score["days_employed"].apply(days_to_months)

print(credit_score["months_employed"])
print(credit_score["months_employed"].isna().sum())
credit_score.isna().sum()

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64
137.46666666666667
187.43333333333334
11464.566666666668

0          281.255768
1          134.160125
2          187.447420
3          137.491574
4        11342.202402
             ...     
21520      150.977222
21521    11464.580138
21522       70.444896
21523      103.749390
21524       66.150253
Name: months_employed, Length: 21525, dtype: float64
2174


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
months_employed     2174
dtype: int64

[Let's now look at the client's age and whether there are any issues there. Again, think about what can data can be strange in this column, i.e. what cannot be someone's age.]

In [95]:
# Check the `dob_years` for suspicious values and count the percentage
age1 = credit_score["dob_years"].value_counts()
print(age1.sort_values())

75      1
74      6
73      8
19     14
72     33
20     51
71     58
70     65
69     85
68     99
0     101
21    111
67    167
22    183
66    183
65    194
23    254
24    264
64    265
63    269
62    352
61    355
25    357
60    377
26    408
55    443
59    444
51    448
53    459
57    460
58    461
46    475
54    479
47    480
52    484
56    487
27    493
45    497
28    503
49    508
32    510
43    513
50    514
37    537
48    538
30    540
29    545
44    547
36    555
31    560
39    573
33    581
42    597
38    598
34    603
41    607
40    609
35    617
Name: dob_years, dtype: int64


[Decide what you'll do with the problematic values and explain why.]
There are 101 rows that report age of 0. In order to even apply for a loan, one must be 18 years or older. Instead of getting rid of these rows, I will try to salvage the data by averaging the age in the dataset and applying it to these rows. I choose to do this in order to save the data that we have in this dataset rather than discarding. After all, I don't anticipate that age will have an impact in whether or not a client will default on a loan, so changing the age to the average age will likely not have a significant impact on the results.

In [96]:
# Address the issues in the `dob_years` column, if they exist
credit_score["dob_years"].mean()
credit_score.loc[credit_score["dob_years"] == 0, "dob_years"] = int(43.27)

In [97]:
# Check the result - make sure it's fixed
print(credit_score["dob_years"].value_counts())

35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


[Now let's check the `family_status` column. See what kind of values there are and what problems you may need to address.]
There do not seem to be any apparent issues with the values in the ["family_status"] column

In [98]:
# Let's see the values for the column
credit_score["family_status"].value_counts()


married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [99]:
# Address the problematic values in `family_status`, if they exist

#I do not see any apparent issues. -Dina

In [100]:
# Check the result - make sure it's fixed
#No issues- Dina

[Now let's check the `gender` column. See what kind of values there are and what problems you may need to address]
There seems to be double the number of females than males in this dataset. I am unsure what "XNA" stands for, but since this gender category only has 1 row out of the entire dataset, it is probably a good idea to just get rid of it. 

In [101]:
# Let's see the values in the column
credit_score["gender"].value_counts()
credit_score.loc[credit_score['gender']=='XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,months_employed
10701,0,2358.600502,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate,78.620017


In [102]:
# Address the problematic values, if they exist
credit_score.drop(10684, axis=0, inplace=True)

In [103]:
# Check the result - make sure it's fixed

credit_score["gender"].value_counts()

F      14235
M       7288
XNA        1
Name: gender, dtype: int64

[Now let's check the `income_type` column. See what kind of values there are and what problems you may need to address]
The issue here is that there is such little data on the following columns : ["unemployed"], ["entrepreneur"], ["paternity/maternity leave" and ["student"].

The goal is to analyze what factors MOST impact a client's likelihood to default on a loan. It is impossible to draw any conclusions about these values given that cumulatively they barely amount to 1% of the whole dataset. I think it is fair to say we can drop these rows.

In [104]:
# Let's see the values in the column
credit_score["income_type"].value_counts()

employee                       11119
business                        5085
retiree                         3855
civil servant                   1459
entrepreneur                       2
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [105]:
# Address the problematic values, if they exist
credit_score=credit_score.drop(credit_score.index[credit_score['income_type']=='unemployed'])
credit_score=credit_score.drop(credit_score.index[credit_score['income_type']=='entrepreneur'])
credit_score=credit_score.drop(credit_score.index[credit_score['income_type']=='paternity / maternity leave'])
credit_score=credit_score.drop(credit_score.index[credit_score['income_type']=='student'])


In [106]:
# Check the result - make sure it's fixed
credit_score["income_type"].value_counts()

employee         11119
business          5085
retiree           3855
civil servant     1459
Name: income_type, dtype: int64

[Now let's see if we have any duplicates in our data. If we do, you'll need to decide what you will do with them and explain why.]

In [107]:
# Checking duplicates
#credit_score.duplicated().sum()
pd.set_option('display.max_rows', 200)
print(credit_score[credit_score.duplicated(keep=False)].sort_values(by=["dob_years", "education", "family_status", "purpose" ]))

       children  days_employed  dob_years            education  education_id  \
8853          1            NaN         23  secondary education             1   
20297         1            NaN         23  secondary education             1   
15892         0            NaN         23  secondary education             1   
19321         0            NaN         23  secondary education             1   
3452          0            NaN         29    bachelor's degree             0   
18328         0            NaN         29    bachelor's degree             0   
8629          1            NaN         30    bachelor's degree             0   
21281         1            NaN         30    bachelor's degree             0   
680           1            NaN         30    bachelor's degree             0   
18349         1            NaN         30    bachelor's degree             0   
4216          0            NaN         30  secondary education             1   
6312          0            NaN         3

In [108]:
# Address the duplicates, if they exist
credit_score[credit_score.duplicated(keep=False)].sort_values(by=["dob_years", "education", "family_status", "purpose" ])
credit_score=credit_score.drop_duplicates().reset_index(drop=True)

In [109]:
# Last check whether we have any duplicates

credit_score[credit_score.duplicated(keep=False)].sort_values(by=["dob_years", "education", "family_status", "purpose" ])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,months_employed


The table is empty above, which indicates that all duplicates have been dropped.

In [110]:
# Check the size of the dataset that you now have after your first manipulations with it
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21447 entries, 0 to 21446
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21447 non-null  int64  
 1   days_employed     19345 non-null  float64
 2   dob_years         21447 non-null  int64  
 3   education         21447 non-null  object 
 4   education_id      21447 non-null  int64  
 5   family_status     21447 non-null  object 
 6   family_status_id  21447 non-null  int64  
 7   gender            21447 non-null  object 
 8   income_type       21447 non-null  object 
 9   debt              21447 non-null  int64  
 10  total_income      19345 non-null  float64
 11  purpose           21447 non-null  object 
 12  months_employed   19345 non-null  float64
dtypes: float64(3), int64(5), object(5)
memory usage: 2.1+ MB


[Describe your new dataset: briefly say what's changed and what's the percentage of the changes, if there were any.]
The new dataset has been reduced by 71 rows, which were all duplicates. The new dataset now contains a total of 21454 rows. This means that .329% of the data has been dropped.

# Working with missing values

[To speed up working with some data, you may want to work with dictionaries for some values, where IDs are provided. Explain why and which dictionaries you will work with.]

In [111]:
# Find the dictionaries
print(dict(zip(credit_score.family_status, credit_score.family_status_id)))
print(dict(zip(credit_score.education, credit_score.education_id)))

{'married': 0, 'civil partnership': 1, 'widow / widower': 2, 'divorced': 3, 'unmarried': 4}
{"bachelor's degree": 0, 'secondary education': 1, 'some college': 2, 'primary education': 3, 'graduate degree': 4}


### Restoring missing values in `total_income`

[Briefly state which column(s) have values missing that you need to address. Explain how you will fix them.]


[Start with addressing total income missing values. Create and age category for clients. Create a new column with the age category. This strategy can help with calculating values for the total income.]


In [112]:
# Let's write a function that calculates the age category
def age_category(age):
    if age <= 18:
        return 'unqualified for loan'
    if 18 <= age <= 25:
        return "18-25"
    if 26 <= age <= 35:
        return "26-35"
    if 36 <= age <= 45:
        return "36-45"
    if 46 <= age <= 55:
        return "46-55"
    if 56 <= age <= 65:
        return "56-65"
    if 66 <= age <=77:
        return "66-77"
        

In [113]:
# Test if the function works
age_category(15)

'unqualified for loan'

The function works!

In [114]:
# Creating new column based on function
credit_score["age_categories"]=credit_score["dob_years"].apply(age_category)


In [115]:
# Checking how values in the new column
credit_score["age_categories"].value_counts()


36-45    5719
26-35    5349
46-55    4814
56-65    3632
18-25    1231
66-77     702
Name: age_categories, dtype: int64

[Think about the factors on which income usually depends. Eventually, you will want to find out whether you should use mean or median values for replacing missing values. To make this decision you will probably want to look at the distribution of the factors you identified as impacting one's income.]

[Create a table that only has data without missing values. This data will be used to restore the missing values.]

In [116]:
# Create a table without missing values and print a few of its rows to make sure it looks fine
#selected_rows = credit_score[~credit_score.isna()]
#print(selected_rows)
selected_rows = credit_score[credit_score != credit_score.isna()]
print(selected_rows)


       children  days_employed  dob_years            education  education_id  \
0           1.0    8437.673028         42    bachelor's degree           NaN   
1           1.0    4024.803754         36  secondary education           1.0   
2           NaN    5623.422610         33  secondary education           1.0   
3           3.0    4124.747207         32  secondary education           1.0   
4           NaN  340266.072047         53  secondary education           1.0   
...         ...            ...        ...                  ...           ...   
21442       1.0    4529.316663         43  secondary education           1.0   
21443       NaN  343937.404131         67  secondary education           1.0   
21444       1.0    2113.346888         38  secondary education           1.0   
21445       3.0    3112.481705         38  secondary education           1.0   
21446       2.0    1984.507589         40  secondary education           1.0   

           family_status  family_status

In [117]:
# Look at the mean values for income based on your identified factors
print(credit_score["total_income"].mean())
print(credit_score["days_employed"].abs().mean())


26787.206510054275
66877.92093159157


In [118]:
# Look at the median values for income based on your identified factors
print(credit_score["total_income"].median())
print(credit_score["days_employed"].abs().median())

23203.786
2194.2169684631217


Since total income varies greatly among the very rich and the very poor, I think it would be more appropriate to use the median because it would account for the outliers.

[Repeat such comparisons for multiple factors. Make sure you consider different aspects and explain your thinking process.]


[Make a decision on what characteristics define income most and whether you will use a median or a mean. Explain why you made this decision]

In [119]:
#  Write a function that we will use for filling in missing values
def fill_in(val):
    median = credit_score["total_income"].median()
    if pd.notnull(val) == False:
        return median
    else: 
        return val 

In [120]:
# Check if it works
#credit_score["total_income"] = credit_score["total_income"].fillna(credit_score["total_income"].median())
#print(fill_in(credit_score["total_income"]))
#print(credit_score.loc[credit_score["total_income"]== float("23203.786"), :])
#print(credit_score.loc[credit_score["total_income"].isna().sum()])
#credit_score["total_income"].apply(fill_in)
print(fill_in(1))
print(fill_in(nd.nan))


                             

1
23203.786


In [121]:
credit_score["total_income"].isna().sum()

2102

In [122]:
# Apply it to every row
credit_score["total_income"] = credit_score["total_income"].apply(fill_in)
print(credit_score["total_income"])

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21442    35966.698
21443    24959.969
21444    14347.610
21445    39054.888
21446    13127.587
Name: total_income, Length: 21447, dtype: float64


In [123]:
print(credit_score.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21447 entries, 0 to 21446
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21447 non-null  int64  
 1   days_employed     19345 non-null  float64
 2   dob_years         21447 non-null  int64  
 3   education         21447 non-null  object 
 4   education_id      21447 non-null  int64  
 5   family_status     21447 non-null  object 
 6   family_status_id  21447 non-null  int64  
 7   gender            21447 non-null  object 
 8   income_type       21447 non-null  object 
 9   debt              21447 non-null  int64  
 10  total_income      21447 non-null  float64
 11  purpose           21447 non-null  object 
 12  months_employed   19345 non-null  float64
 13  age_categories    21447 non-null  object 
dtypes: float64(3), int64(5), object(6)
memory usage: 2.3+ MB
None


In [124]:
# Check if we got any errors
credit_score.isna().sum()

children               0
days_employed       2102
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
months_employed     2102
age_categories         0
dtype: int64

-Note that the function was applied to the ["total_income"] column and all missing values for this column have been filled in. The cell above shows that there are no more missing values for this particular column. Next, I will do the same thing to the ["days_employed"] column.

[If you've came across errors in preparing the values for missing data, it probably means there's something special about the data for the category. Give it some thought - you may want to fix some things manually, if there's enough data to find medians/means.]

In [125]:
# Replacing missing values if there are any errors
#No errors found -Dina

[When you think you've finished with `total_income`, check that the total number of values in this column matches the number of values in other ones.]

In [126]:
# Checking the number of entries in the columns
credit_score.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21447 entries, 0 to 21446
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21447 non-null  int64  
 1   days_employed     19345 non-null  float64
 2   dob_years         21447 non-null  int64  
 3   education         21447 non-null  object 
 4   education_id      21447 non-null  int64  
 5   family_status     21447 non-null  object 
 6   family_status_id  21447 non-null  int64  
 7   gender            21447 non-null  object 
 8   income_type       21447 non-null  object 
 9   debt              21447 non-null  int64  
 10  total_income      21447 non-null  float64
 11  purpose           21447 non-null  object 
 12  months_employed   19345 non-null  float64
 13  age_categories    21447 non-null  object 
dtypes: float64(3), int64(5), object(6)
memory usage: 2.3+ MB


###  Restoring values in `days_employed`

[Think about the parameters that may help you restore the missing values in this column. Eventually, you will want to find out whether you should use mean or median values for replacing missing values. You will probably conduct a research similar to the one you've done when restoring data in a previous column.]

In [127]:
# Distribution of `days_employed` medians based on your identified parameters
print(credit_score["days_employed"].abs().median())
credit_score["days_employed"].value_counts().sort_values()


2194.2169684631217


142.276217     1
382.621465     1
1578.419429    1
1214.614264    1
2793.758190    1
              ..
4665.860899    1
1513.814188    1
1042.200490    1
3229.233747    1
1636.419775    1
Name: days_employed, Length: 19345, dtype: int64

In [128]:
# Distribution of `days_employed` means based on your identified parameters
print(credit_score["days_employed"].mean())

66877.92093159157


[Decide what you will use: means or medians. Explain why.]
For this column, I choose to use the median as well because there is such a large gap between the minimum and maximum values for this column. I would anticipate that there would be many outliers in this column because of that. The median is a more resistant measure of central tendancy to outliers than the mean.

In [129]:
# Let's write a function that calculates means or medians (depending on your decision) based on your identified parameter
def fill_in(val):
    median = credit_score["days_employed"].median()
    if pd.notnull(val) == False:
        return median
    else: 
        return val 

In [130]:
# Check that the function works
print(fill_in(1))
print(fill_in(nd.nan))

1
2194.2169684631217


In [131]:
# Apply function to the income_type
credit_score["days_employed"] = credit_score["days_employed"].apply(fill_in)

In [132]:
# Check if function worked
credit_score["days_employed"].isna().sum()


0

Note that there are no more missing values in the ["days_employed"] column. All of the missing values have been filled in with the median.

Confirmed- No more missing values

[When you think you've finished with `total_income`, check that the total number of values in this column matches the number of values in other ones.]
The ["days_employed"] has been restored and all of the missing values in that column have been replaced with the median for the reasons mentioned above.

In [133]:
# Check the entries in all columns - make sure we fixed all missing values
print(credit_score.isna().sum())

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
months_employed     2102
age_categories         0
dtype: int64


This also shows that all missing values are taken care of.

## Categorization of data

[To answer the questions and test the hypotheses, you will want to work with categorized data. Look at the questions that were posed to you and that you should answer. Think about which of the data will need to be categorized to answer these questions. Below you will find a template through which you can work your way when categorizing data. The first step-by-step processing covers the text data; the second one addresses the numerical data that needs to be categorized. You can use both or none of the suggested instructions - it's up to you.]

[Despite of how you decide to address the categorization, make sure to provide clear explanation of why you made your decision. Remember: this is your work and you make all decisions in it.]


In [134]:
# Print the values for your selected data for categorization
print(credit_score["purpose"].value_counts())



wedding ceremony                            791
having a wedding                            767
to have a wedding                           765
real estate transactions                    675
buy commercial real estate                  661
housing transactions                        652
transactions with commercial real estate    650
buying property for renting out             650
purchase of the house                       646
housing                                     646
purchase of the house for my family         638
construction of own property                634
property                                    633
transactions with my real estate            627
building a real estate                      624
buy real estate                             621
purchase of my own house                    620
building a property                         619
housing renovation                          606
buy residential real estate                 605
buying my own car                       

[Let's check unique values]

In [135]:
# Check the unique valuesprint
print(credit_score["purpose"].unique())

['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 'purchase of a car'
 'profile education' 'university education'
 'buying property for renting out' 'to buy a car' 'housing renovation'
 'going to university']


[What main groups can you identify based on the unique values?]
The following six main groups that I see are truly "unique":
-Car-related purchase
-House-related purchase
-Wedding 
-Real Estate
-Education
 

[Based on these themes, we will probably want to categorize our data.]


In [136]:
# Let's write a function to categorize the data based on common topics
def purpose_group(value):
    if "house" in value or "housing" in value:
        return "House Purchase"
    elif "real estate" in value or "property" in value:
        return "Real Estate Purchase"
    elif "education" in value or "educated" in value or "university" in value:
        return "Education"
    elif 'wedding' in value:
        return "Wedding"
    elif "car" in value or "cars" in value:
        return "Car Purchase"
   





In [137]:
# Create a column with the categories and count the values for them
print(purpose_group('purchase of the house'))
print(purpose_group("real estate"))
print(purpose_group('to own a car'))
print(purpose_group('getting higher education'))
print(purpose_group('to have a wedding'))
print()
credit_score["purpose_group"] = credit_score["purpose"].apply(purpose_group)
print()
print(credit_score["purpose_group"])

print()

print(credit_score["purpose_group"].value_counts())


House Purchase
Real Estate Purchase
Car Purchase
Education
Wedding


0              House Purchase
1                Car Purchase
2              House Purchase
3                   Education
4                     Wedding
                 ...         
21442          House Purchase
21443            Car Purchase
21444    Real Estate Purchase
21445            Car Purchase
21446            Car Purchase
Name: purpose_group, Length: 21447, dtype: object

Real Estate Purchase    6999
Car Purchase            4304
Education               4013
House Purchase          3808
Wedding                 2323
Name: purpose_group, dtype: int64


[If you decide to categorize the numerical data, you'll need to come up with the categories for it too.]
In this dataset, I think the only numeric column that would be useful to categorize would be 

In [138]:
# Looking through all the numerical data in your selected column for categorization
print(credit_score["children"].value_counts())
print()
print(credit_score["total_income"].sort_values(ascending=False))
print(credit_score["total_income"].max())

def total_income_group(val):
    if val <= 50000:
        return "Low income: $3,000 to $40,000"
    elif val <= 80000:
        return "Medium income: $50,000 to $80,000"
    elif val <= 370000:
        return "High income: $90,000 to 370,000"
    elif val > 370000:
        return "Very high income: Above $370,000"
    
    
    
print(total_income_group(285000))
print()
print(total_income_group(80000))

credit_score["total_income_group"] = credit_score["total_income"].apply(total_income_group)
print(credit_score["total_income_group"].value_counts())
print(credit_score.head(30))

0     14086
1      4854
2      2051
3       330
20       76
4        41
5         9
Name: children, dtype: int64

12386    362496.645
19542    352136.354
9157     276204.162
20736    274402.943
17132    273809.483
            ...    
14243      3503.298
1598       3471.216
16133      3418.824
12979      3392.845
14551      3306.762
Name: total_income, Length: 21447, dtype: float64
362496.645
High income: $90,000 to 370,000

Medium income: $50,000 to $80,000
Low income: $3,000 to $40,000        20128
Medium income: $50,000 to $80,000     1097
High income: $90,000 to 370,000        222
Name: total_income_group, dtype: int64
    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42    bachelor's degree             0   
1          1    4024.803754         36  secondary education             1   
2          0    5623.422610         33  secondary education             1   
3          3    4124.747207         32  secondary education   

In [139]:
# Getting summary statistics for the column
print(credit_score["children"].describe())
one_or_more_kids = credit_score[credit_score["children"] >= 1].count()
no_kids = credit_score[credit_score["children"] == 0].count()
print()
print(one_or_more_kids / len(credit_score))
print()
print(no_kids / len(credit_score))


count    21447.000000
mean         0.544365
std          1.381887
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

children              0.343218
days_employed         0.343218
dob_years             0.343218
education             0.343218
education_id          0.343218
family_status         0.343218
family_status_id      0.343218
gender                0.343218
income_type           0.343218
debt                  0.343218
total_income          0.343218
purpose               0.343218
months_employed       0.309554
age_categories        0.343218
purpose_group         0.343218
total_income_group    0.343218
dtype: float64

children              0.656782
days_employed         0.656782
dob_years             0.656782
education             0.656782
education_id          0.656782
family_status         0.656782
family_status_id      0.656782
gender                0.656782
income_type           0.656782
d

The cell above indicates that in this dataset, the majority of customers have 0 kids (65%), while the remaining 34% of customers in this dataset have 1 or more children. I choose to compare two groups of data in the ["children"] column: those with no kids versus those with 1 or more children. I choose not to split them into more groups because that would split the already low 34% of data into smaller portions, which would not be optimal when trying to draw significant conclusions.

[Decide what ranges you will use for grouping and explain why.]
According to the summary statistics of the ["children"] column, the majority of individuals in this dataset have 0 kids. About 25% of individuals have 1 or more kids.

In [140]:
# Creating function for categorizing into different numerical groups based on ranges
def number_of_children(val):
    if val == 0:
        return "No children"
    elif val >= 1:
        return "Has children"
        
print(number_of_children(0))
print(number_of_children(1))
print(number_of_children(2))
print(number_of_children(5))



No children
Has children
Has children
Has children


In [141]:
# Creating column with categories
credit_score["number_of_children"] = credit_score["children"].apply(number_of_children)
print(credit_score["number_of_children"])

0        Has children
1        Has children
2         No children
3        Has children
4         No children
             ...     
21442    Has children
21443     No children
21444    Has children
21445    Has children
21446    Has children
Name: number_of_children, Length: 21447, dtype: object


In [142]:
# Count each categories values to see the distribution
print(credit_score["number_of_children"].value_counts())


No children     14086
Has children     7361
Name: number_of_children, dtype: int64


## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

In [143]:
# Check the children data and paying back on time
effect_of_children_on_debt = credit_score.groupby(["number_of_children", "debt"]).size().reset_index(name="count")
print(effect_of_children_on_debt)
print()

# Calculating default-rate based on the number of children

total_default = sum(effect_of_children_on_debt[effect_of_children_on_debt["debt"] == 1]["count"])
print("Total Default Count:", total_default )

print()


no_children_default_rate = effect_of_children_on_debt["count"][3] / total_default
print("No Children Default Rate:", no_children_default_rate)
percentage = "{:.1%}".format(no_children_default_rate)
print("No Children Default Rate:", percentage)

print()

has_children_default_rate = effect_of_children_on_debt["count"][1] / total_default
print("Has Children Default Rate:", has_children_default_rate)
percentage2 = "{:.1%}".format(has_children_default_rate)
print("Has Children Default Rate:",percentage2 )


  number_of_children  debt  count
0       Has children     0   6685
1       Has children     1    676
2        No children     0  13023
3        No children     1   1063

Total Default Count: 1739

No Children Default Rate: 0.6112708453133985
No Children Default Rate: 61.1%

Has Children Default Rate: 0.3887291546866015
Has Children Default Rate: 38.9%


**Conclusion**

[Write your conclusions based on your manipulations and observations.]
Based on the results, it appears that having no children resulted in a higher default rate (61.1%) than those who reported having children (38.9%). Given the large difference between the two categories, this may suggest that there may be a correlation between whether or not an individual has children and their likelihood to default on a loan.


In [145]:
credit_score.pivot_table(index=["number_of_children"], values= ["debt"], aggfunc="mean", margins=True)

,debt
number_of_children,
Has children,0.091835
No children,0.075465
All,0.081084


**Is there a correlation between family status and paying back on time?**

In [154]:
# Check the family status data and paying back on time+
effect_of_family_status_on_debt = credit_score.groupby(["family_status", "debt"]).size().reset_index(name="count")
print(effect_of_family_status_on_debt)

print()

# Calculating default-rate based on family status
total_default = sum(effect_of_children_on_debt[effect_of_children_on_debt["debt"] == 1]["count"])
print("Total Default Count:", total_default)

print()

civil_partnership_default_rate = effect_of_family_status_on_debt["count"][1] / total_default
print("Civil Partnership:", civil_partnership_default_rate)
percentage3 = "{:.1%}".format(civil_partnership_default_rate)
print("Civil Partnership:", percentage3)
print()
divorced_default_rate =effect_of_family_status_on_debt["count"][3] / total_default
print("Divorced:", divorced_default_rate)
percentage4 = "{:.1%}".format(divorced_default_rate)
print("Divorced:", percentage4)
print()
married_default_rate = effect_of_family_status_on_debt["count"][5] / total_default
print("Married:", married_default_rate)
percentage5 = "{:.1%}".format(married_default_rate)
print("Married:",percentage5)
print()
unmarried_default_rate = effect_of_family_status_on_debt["count"][7] / total_default
print("Unmarried:", unmarried_default_rate)
percentage6 = "{:.1%}".format(unmarried_default_rate)
print("Unmarried:", percentage6)
print()
widow_widower_default_rate = effect_of_family_status_on_debt["count"][9] / total_default
print("Widower/Widow:", widow_widower_default_rate)
percentagex= "{:.1%}".format(widow_widower_default_rate)
print("Widower/Widow:", percentagex)


credit_score.pivot_table(index=["family_status"], values= ["debt"], aggfunc="mean")

       family_status  debt  count
0  civil partnership     0   3761
1  civil partnership     1    388
2           divorced     0   1109
3           divorced     1     85
4            married     0  11407
5            married     1    929
6          unmarried     0   2535
7          unmarried     1    274
8    widow / widower     0    896
9    widow / widower     1     63

Total Default Count: 1739

Civil Partnership: 0.22311673375503163
Civil Partnership: 22.3%

Divorced: 0.0488786658999425
Divorced: 4.9%

Married: 0.5342150661299597
Married: 53.4%

Unmarried: 0.15756181713628523
Unmarried: 15.8%

Widower/Widow: 0.03622771707878091
Widower/Widow: 3.6%


,debt
family_status,
civil partnership,0.093517
divorced,0.071189
married,0.075308
unmarried,0.097544
widow / widower,0.065693


***Note to to the reviewer: I attempted to use pivot tables to calculate the default rates, but they did not match the default rates I calculated manually. I reached out to my tutor, and he expressed that the method I used was sufficient and said that I would need to somehow subset the df in order to obtain pivot tables with the correct rates. However, I tried to subset the dataframe so that it narrowed down to just defaults, but that attempt was not successful either. Please advise.

**Conclusion**

[Write your conclusions based on your manipulations and observations.]
Out of all the other family status categories, individuals with "married" as their family status have the highest default rate at 53.4%. "Civil Partnership" comes in as second most likely to default at a rate of 22.3%. "Unmarried" family status resulted in a default rate of 15.8%. "Divorced" family status resulted in a 4.9% default rate. The family status label that resulted in the lowest defualt rate was "Widow/Widower", at only 3.6%. This suggests that there may be a correlation between family status and defaulting on a loan, but further tests must be conducted in order to confirm this.

**Is there a correlation between income level and paying back on time?**

In [72]:
# Check the income level data and paying back on time
total_default = sum(effect_of_children_on_debt[effect_of_children_on_debt["debt"] == 1]["count"])
print("Total Default Count:", total_default)

print()

effect_of_income_level_on_debt = credit_score.groupby(["income_type", "debt"]).size().reset_index(name="count")
print(effect_of_income_level_on_debt)
print()
business_income_default_rate = effect_of_income_level_on_debt["count"][1] / total_default
print(business_income_default_rate)
percentage10 = "{:.1%}".format(business_income_default_rate)
print("Business Income Type:",percentage10)
print()
civil_servant_income_default_rate = effect_of_income_level_on_debt["count"][3] / total_default
print(civil_servant_income_default_rate)
percentage11 = "{:.1%}".format(civil_servant_income_default_rate)
print("Civil Servant Income Type:", percentage11)
print()
employee_income_default_rate = effect_of_income_level_on_debt["count"][5] / total_default
print(employee_income_default_rate)
percentage12 = "{:.1%}".format(employee_income_default_rate)
print("Employee Income Type:", percentage12)
print()
retiree_income_default_rate = effect_of_income_level_on_debt["count"][7] / total_default
print(retiree_income_default_rate)
percentage13 = "{:.1%}".format(retiree_income_default_rate)
print("Retiree Income Type:", percentage13)
print()
# Calculating default-rate based on income level

print()

effect_of_total_income_on_debt = credit_score.groupby(["total_income_group", "debt"]).size().reset_index(name="count")
print(effect_of_total_income_on_debt)

print()

high_income_default_rate = effect_of_total_income_on_debt["count"][1] / total_default
print(high_income_default_rate)
percentage7 = "{:.1%}".format(high_income_default_rate)
print("High Income Default Rate:",percentage7)
print()
medium_income_default_rate = effect_of_total_income_on_debt["count"][5] / total_default
print(medium_income_default_rate)
percentage8 = "{:.1%}".format(medium_income_default_rate)
print("Medium Income Default_Rate:", percentage8)
print()
low_income_default_rate = effect_of_total_income_on_debt["count"][3] / total_default
print(low_income_default_rate)
percentage9 = "{:.1%}".format(low_income_default_rate)
print("Low Income Default_Rate:", percentage9)

credit_score.pivot_table(index=["total_income_group"], values= ["debt"], aggfunc="mean", margins=True)

Total Default Count: 1739

     income_type  debt  count
0       business     0   4702
1       business     1    376
2  civil servant     0   1371
3  civil servant     1     86
4       employee     0  10023
5       employee     1   1061
6        retiree     0   3612
7        retiree     1    216

0.21621621621621623
Business Income Type: 21.6%

0.04945370902817711
Civil Servant Income Type: 4.9%

0.6101207590569293
Employee Income Type: 61.0%

0.1242093156986774
Retiree Income Type: 12.4%


                  total_income_group  debt  count
0    High income: $90,000 to 370,000     0    208
1    High income: $90,000 to 370,000     1     14
2      Low income: $3,000 to $40,000     0  18481
3      Low income: $3,000 to $40,000     1   1647
4  Medium income: $50,000 to $80,000     0   1019
5  Medium income: $50,000 to $80,000     1     78

0.008050603795284647
High Income Default Rate: 0.8%

0.044853364002300174
Medium Income Default_Rate: 4.5%

0.9470960322024152
Low Income Default_Rate: 9

,debt
total_income_group,
"High income: $90,000 to 370,000",0.063063
"Low income: $3,000 to $40,000",0.081826
"Medium income: $50,000 to $80,000",0.071103
All,0.081084


**Conclusion**

[Write your conclusions based on your manipulations and observations.]
The default rates based on income indicate that individuals in the low income category have the highest default rate (94.7%) as compared to the medium income category (4.5%). The high income category resulted in the lowest default rate (0.8%). These outcomes are expected, as one would think that the more money you make, the more likely one would pay back the loan on time. Furthermore, the data indicates that individuals who have reported "employee" type income also have the highest default rate at 61.0%, while the individuals with a "business" income type resulted in a rate of 21.6%. "Retiree" income type resulted in a 12.4% rate. The lowest default rate of all income types was found to be "civil servant" type at a rate of only 4.9%. The large difference in default rates among the [total_income categories] most likely suggests that there may be a considerable correlation between the amount of money someone makes and their likelihood to default on a loan. Further statistical testing must be conducted in order to confirm this correlation.

**How does credit purpose affect the default rate?**

In [73]:
# Check the percentages for default rate for each credit purpose and analyze them
effect_of_purpose_on_debt = credit_score.groupby(["purpose_group", "debt"]).size().reset_index(name="count")
print(effect_of_purpose_on_debt)
print()
total_default = sum(effect_of_children_on_debt[effect_of_children_on_debt["debt"] == 1]["count"])
print("Total Default Count:", total_default)
print()
car_purchase_default_rate = effect_of_purpose_on_debt["count"][1] / total_default
print(car_purchase_default_rate)
percentage15 = "{:.1%}".format(car_purchase_default_rate)
print("Car Purchase Default Rate:",percentage15)
print()
education_default_rate = effect_of_purpose_on_debt["count"][3] / total_default
print(education_default_rate)
percentage16 = "{:.1%}".format(education_default_rate )
print("Education Default Rate:", percentage16)
print()
house_purchase_default_rate = effect_of_purpose_on_debt["count"][5] / total_default
print(house_purchase_default_rate)
percentage17 = "{:.1%}".format(house_purchase_default_rate)
print("House Purchase Default Rate:", percentage17)
print()
real_estate_default_rate = effect_of_purpose_on_debt["count"][7] / total_default
print(real_estate_default_rate)
percentage18 = "{:.1%}".format(real_estate_default_rate)
print("Real Estate Default Rate:", percentage18)
print()
wedding_purchase_default_rate = effect_of_purpose_on_debt["count"][9] / total_default
print(wedding_purchase_default_rate)
percentage19 = "{:.1%}".format(wedding_purchase_default_rate)
print("Wedding Purchase Default Rate:", percentage19)

credit_score.pivot_table(index=["purpose_group"], values= ["debt"], aggfunc="mean", margins=True)

          purpose_group  debt  count
0          Car Purchase     0   3902
1          Car Purchase     1    402
2             Education     0   3643
3             Education     1    370
4        House Purchase     0   3552
5        House Purchase     1    256
6  Real Estate Purchase     0   6474
7  Real Estate Purchase     1    525
8               Wedding     0   2137
9               Wedding     1    186

Total Default Count: 1739

0.23116733755031627
Car Purchase Default Rate: 23.1%

0.2127659574468085
Education Default Rate: 21.3%

0.1472110408280621
House Purchase Default Rate: 14.7%

0.30189764232317423
Real Estate Default Rate: 30.2%

0.10695802185163887
Wedding Purchase Default Rate: 10.7%


,debt
purpose_group,
Car Purchase,0.093401
Education,0.092200
House Purchase,0.067227
Real Estate Purchase,0.075011
Wedding,0.080069
All,0.081084


**Conclusion**

Unlike the previous factors that have been examined above, default rates based on purpose do not appear to possess the same sharp difference between different purpose groups. In other words, there doesn't seem to be a clear indication that credit purpose correlates with defaulting on a loan. The credit purpose that resulted in the highest default rate was "real estate" at 30.2%. The credit purpose of "car purchase" and "education" resulted in fairly similar default rates, 23.1% and 21.3% respectively. Similarly, the "wedding purchase" and "house purchase" categories were also close at 10.7% and 14.7% respectively.

# General Conclusion 

[List your conclusions in this final section. Make sure you include all your important conclusions you made that led you to the way you processed and analyzed the data. Cover the missing values, duplicates, and possible reasons and solutions for problematic artifacts that you had to address.]

[List your conclusions regarding the posed questions here as well.]


In this dataset, there were several aspects to consider. This dataset consisted of about 20% of missing data in accross all columns. All of the missing values found in the [days_employed] column and the [total_income] column were filled in with their respective medians. The median was chosed to fill in the missing values due to the high number of outliers in both these columns. Thus, the mean would not have been an appropriate measure to utilize in this case. About 1% of the dataset contained duplicates, which were consequently dropped. This dataset also conisted of some problematic data, which was either converted or dropped. A large set of negative values were found in the [days_employed] column, which were converted to positive values, as it is likely that the negative sign placed in front of these values was due to error. I also created the following additional columns in order to better categorize the data in the columns which included: [months_employed], [purpose_group], [total_income_group], [number_of_children], a      In addition, there were a set of values in the [dob_years] column that indicated "0" as a value. This might have also been an error, as one cannot apply for a loan until the age of 18. Thus, rather than dropping the values, they were replaced with the mean of the [dob_years] column. Furthermore, the number of [children] column contained a set of rows with a value of "-1", which clearly does not make sense. This appeared to have the same mistake as the [days_employed] column with a negative sign incorrectly placed in front of the value. Instead of dropping these rows, it is likley that the intended value here was actually "1", so I changed the value to "1" instead of "-1". In any case, even if the value should have been some other number, those rows amounted to less than 1% of the data, so it would not have changed anything in the later analysis.



Is there a correlation between having children and paying back on time?
Yes, it is likely that there is a relationship between having children and paying back on time, as having no children resulted in a much higher default rate (61.1%) as compared to having children (38.9%).

Is there a correlation between income level and paying back on time?
Yes, it appears that having a high income makes it dramatically less likely to default on a loan (0.8%) than having a low income, as the rate for low income resulted in a 94.7% default rate! Interestingly, having a medium income resulted in a low default rate as well (4.5%).

Is there a correlation between family status and paying back on time?
Family status does seem to impact the default rate in some way, as having the "married" family status resulted in the highest default rate (53.4%). The rest of the family_status categories resuled in the following:
Civil Partnership: 22.3%, Divorced: 4.9%, Unmarried: 15.8%, and Widower/Widow: 3.6%.

How does credit purpose affect the default rate?
Credit purpose overall does not seem to be strongly correlated with paying back on time, as all of the categories within [credit_purose] deviate on a smaller scale than factors such as income level and having children:
Car Purchase Default Rate: 23.1%, Education Default Rate: 21.3%, House Purchase Default Rate: 14.7% Real Estate Default Rate: 30.2%, and Wedding Purchase Default Rate: 10.7%.

Further statistical testing should be conducted in order to confirm these suspected correlations.



